In [18]:
import os

import blobconverter
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO

In [19]:
DATA_YAML = "datasets/red_cube/data.yaml"
BASE_WEIGHTS = "models/yolo11n_base/yolo11n.pt"

PROJECT = "runs/detect"
NAME = "red_cube_ultra_simple"

EPOCHS = 50
IMGSZ = 640
BATCH = 8
DEVICE = 0  # "cpu" si pas de GPU
CONF = 0.25

In [20]:
model = YOLO(BASE_WEIGHTS)

results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=DEVICE,
    project=PROJECT,
    name=NAME,
    exist_ok=True,
)

save_dir = getattr(results, "save_dir", None)
if save_dir is None and hasattr(model, "trainer"):
    save_dir = model.trainer.save_dir

best_pt = os.path.join(save_dir, "weights", "best.pt")
last_pt = os.path.join(save_dir, "weights", "last.pt")
print("Best:", best_pt)
print("Last:", last_pt)

Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/red_cube/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=models/yolo11n_base/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=red_cube_ultra_simple, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=

In [21]:
best_model = YOLO(best_pt)

test_img = "datasets/red_cube/valid/images/IMG_0408.JPG"

res = best_model.predict(source=test_img, conf=CONF, imgsz=IMGSZ, verbose=False)[0]

im = cv2.imread(test_img)
assert im is not None, f"Image introuvable: {test_img}"

names = best_model.names

if res.boxes is not None and len(res.boxes) > 0:
    xyxy = res.boxes.xyxy.cpu().numpy()
    conf = res.boxes.conf.cpu().numpy()
    cls = res.boxes.cls.cpu().numpy().astype(int)

    for (x1, y1, x2, y2), c, k in zip(xyxy, conf, cls):
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        label = names[k] if isinstance(names, (list, tuple)) else names.get(k, str(k))
        txt = f"{label} {c:.2f}"

        cv2.rectangle(im, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(im, txt, (x1, max(0, y1 - 8)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

<Figure size 800x800 with 1 Axes>

In [22]:
metrics = best_model.val(
    data=DATA_YAML,
    imgsz=IMGSZ,
    device=DEVICE,
    conf=0.001,
    iou=0.6,
    plots=True,
)

print("Save dir:", metrics.save_dir)

try:
    print("mAP50-95:", float(metrics.box.map))
    print("mAP50   :", float(metrics.box.map50))
    print("mAP75   :", float(metrics.box.map75))
    print("Precision:", float(metrics.box.mp))
    print("Recall   :", float(metrics.box.mr))
except Exception as e:
    print("Impossible de lire les attributs standard (version ultralytics differente).")
    print("Objet metrics:", metrics)
    print("Erreur:", e)

Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 7806MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4711.8±1939.6 MB/s, size: 1364.4 KB)
val: Scanning /home/nathan/Documents/GitHub/3d_edge_object_detection/datasets/red_cube/valid/labels.cache... 8 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 8/8 20.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.0s/it 2.0s
                   all          8          8      0.983          1      0.995      0.938
Speed: 0.7ms preprocess, 2.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/val13
Save dir: /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/val13
mAP50-95: 0.9377924999999999
mAP50   : 0.995
mAP75   : 0.995
Precision: 0.9825731072308085
Recall   : 1.0


In [23]:
onnx_path = best_model.export(
    format="onnx",
    imgsz=IMGSZ,
    opset=13,
    simplify=True,
    dynamic=False,
)

print("ONNX exporte:", onnx_path)

Ultralytics 8.3.241 🚀 Python-3.11.14 torch-2.9.1+cu128 CPU (Intel Core i9-14900HX)

PyTorch: starting from '/home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (5.2 MB)

ONNX: starting export with onnx 1.20.0 opset 13...
ONNX: slimming with onnxslim 0.1.80...
ONNX: export success ✅ 1.4s, saved as '/home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights/best.onnx' (10.1 MB)

Export complete (1.6s)
Results saved to /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights
Predict:         yolo predict task=detect model=/home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights/best.onnx i

In [24]:
blob_path = blobconverter.from_onnx(
    model=onnx_path,
    data_type="FP16",
    shaves=6,
    output_dir=os.path.join(save_dir, "weights"),
)

print("BLOB:", os.path.join(save_dir, "weights"))

[==================================================]
Done
BLOB: /home/nathan/Documents/GitHub/3d_edge_object_detection/runs/detect/red_cube_ultra_simple/weights
